# Interactive fitting of critical point data for ammonia + water with GERG and invariant reducing functions

No departure term...

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
%matplotlib inline
import timeit
import pandas
import matplotlib.pyplot as plt, numpy as np
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 100
import teqp

In [ ]:
names = ['Water','Ammonia']
model = teqp.build_multifluid_model(names, '../mycp', '../mycp/dev/mixtures/mixture_binary_pairs.json',{'estimate':'Lorentz-Berthelot'})
puremodels = [teqp.build_multifluid_model([name], '../mycp', '../mycp/dev/mixtures/mixture_binary_pairs.json') for name in names]
        
def get_mutant(params):
    """ Build a teqp-based mutant from the model parameters """
    if 'type' not in params:
        raise KeyError('type must be provided')

    if params['type'] == 'invariant':
        s = {
            "0":{
                "1": {
                    "BIP":{
                        "type": "invariant",
                        "lambdaT": params['lambdaT'],
                        "phiT": params['phiT'],
                        "lambdaV": params['lambdaV'],
                        "phiV": params['phiV'],
                        "Fij": 0.0
                    },
                    "departure":{
                        "type" : "none"
                    }
                }
            }
        }
        return teqp.build_multifluid_mutant(model, s)
    elif params['type'] == 'GERG':
        s = {
            "0":{
                "1": {
                    "BIP":{
                        "type": "GERG2004",
                        "betaT": params['betaT'],
                        "gammaT": params['gammaT'],
                        "betaV": params['betaV'],
                        "gammaV": params['gammaV'],
                        "Fij": 0.0
                    },
                    "departure":{
                        "type" : "none"
                    }
                }
            }
        }
        return teqp.build_multifluid_mutant(model, s)
    else:
        raise KeyError("Bad type for get_mutant")

In [ ]:
def get_critical_curve(params):
    """ Trace the critical curve and return values """    

    tweaked = get_mutant(params)
    # print(tweaked.get_meta())

    basemodel = None
    tic = timeit.default_timer()
    Tcvec = basemodel.get_Tcvec() if basemodel else model.get_Tcvec()
    rhocvec = 1/basemodel.get_vcvec() if basemodel else 1/model.get_vcvec()
    k = 1 # Have to start at pure second component for now... In theory either are possible.
    T0 = Tcvec[k]
    rho0 = rhocvec
    rho0[1-k] = 0
    curveJSON = tweaked.trace_critical_arclength_binary(T0, rho0, "")
    toc = timeit.default_timer()
#     print(toc-tic, 'sec. to trace critical locus')
    df = pandas.DataFrame(curveJSON)
    df['z0 / mole frac.'] = df['rho0 / mol/m^3']/(df['rho0 / mol/m^3']+df['rho1 / mol/m^3'])
    def add_splus(row):
        rhovec = np.array([row['rho0 / mol/m^3'], row['rho1 / mol/m^3']])
        return tweaked.get_splus(T=row['T / K'], rhovec=rhovec)
    df['s^+'] = df.apply(add_splus,axis=1)
    df['rho / mol/m^3'] = df['rho0 / mol/m^3']+df['rho1 / mol/m^3']
    # df.info()
    return df

def crit_curve_plotter(**kwargs):
    df = get_critical_curve(kwargs)
    plt.plot(df['T / K'], df['p / Pa'])
    
    d = pandas.read_csv('NH3H2Odata.csv', sep=';')
    d = d[d.type == 'PTcrit']
    plt.plot(d['T (K)'], d['p (Pa)'], 'o')
    
    plt.gca().set(xlabel='$T$ / K', ylabel='$p$ / Pa')
    

In [ ]:
crit_curve_plotter(**{'lambdaT': 0.07999999999999999, 'phiT': 1.05, 'lambdaV': -0.1, 'phiV': 0.9800000000000001, 'type': 'invariant'})

interactive_plot = interactive(crit_curve_plotter, lambdaT=(-0.1,0.1,0.02),phiT=(0.9,1.5,0.02),lambdaV=(-0.1,0.1,0.02),phiV=(0.9,1.5,0.02), type=['invariant'])
output = interactive_plot.children[-1]
output.layout.height = '400px'
interactive_plot

In [ ]:
# crit_curve_plotter(**{'betaT': 1.0, 'gammaT': 1.0, 'betaV': 1.0, 'gammaV': 1, 'type': 'GERG'})

interactive_plot = interactive(crit_curve_plotter, betaT=(0.9,1.1,0.01),gammaT=(0.5,2,0.02),betaV=(0.9,1.1,0.02),gammaV=(0.2,2,0.02), type=['GERG'])
output = interactive_plot.children[-1]
output.layout.height = '400px'
interactive_plot